In [ ]:

Federated Learning and Privacy-Preserving AI


In [1]:

import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim

# Simple Neural Network for classification
class SimpleModel(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(SimpleModel, self).__init__()
        self.fc = nn.Linear(input_dim, output_dim)

    def forward(self, x):
        return self.fc(x)

# Synthetic data generation for clients
def generate_client_data(num_clients, samples_per_client, input_dim):
    client_data = []
    for _ in range(num_clients):
        # Generate random data and labels for each client
        data = np.random.randn(samples_per_client, input_dim)
        labels = (data.sum(axis=1) > 0).astype(int)  # Simple binary labels
        client_data.append((torch.tensor(data, dtype=torch.float32),
                            torch.tensor(labels, dtype=torch.long)))
    return client_data

# Train a local model
def train_local_model(model, data, labels, epochs, lr):
    optimizer = optim.SGD(model.parameters(), lr=lr)
    criterion = nn.CrossEntropyLoss()
    for _ in range(epochs):
        optimizer.zero_grad()
        outputs = model(data)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
    return model.state_dict()  # Return trained model parameters

# Aggregate updates with Differential Privacy
def aggregate_models(global_model, client_updates, noise_scale=0.1):
    global_state = global_model.state_dict()
    for key in global_state:
        # Average the client updates and add Gaussian noise for privacy
        updates = torch.stack([client_update[key] for client_update in client_updates])
        global_state[key] = updates.mean(dim=0) + torch.normal(0, noise_scale, size=global_state[key].shape)
    global_model.load_state_dict(global_state)

# Federated learning simulation
def federated_learning(num_clients, rounds, input_dim, output_dim, samples_per_client):
    # Initialize global model
    global_model = SimpleModel(input_dim, output_dim)
    client_data = generate_client_data(num_clients, samples_per_client, input_dim)

    for round in range(rounds):
        print(f"Round {round + 1}")
        client_updates = []

        # Train local models
        for client_id, (data, labels) in enumerate(client_data):
            local_model = SimpleModel(input_dim, output_dim)
            local_model.load_state_dict(global_model.state_dict())  # Start from global model
            local_update = train_local_model(local_model, data, labels, epochs=1, lr=0.01)
            client_updates.append(local_update)

        # Aggregate updates with differential privacy
        aggregate_models(global_model, client_updates, noise_scale=0.1)

        # Evaluate global model on a synthetic test set
        test_data, test_labels = generate_client_data(1, 100, input_dim)[0]
        with torch.no_grad():
            predictions = global_model(test_data).argmax(dim=1)
            accuracy = (predictions == test_labels).float().mean().item()
        print(f"Global Model Accuracy: {accuracy:.2%}")

# Parameters
num_clients = 5
rounds = 10
input_dim = 10
output_dim = 2
samples_per_client = 50

# Run Federated Learning
federated_learning(num_clients, rounds, input_dim, output_dim, samples_per_client)



Round 1
Global Model Accuracy: 44.00%
Round 2
Global Model Accuracy: 53.00%
Round 3
Global Model Accuracy: 63.00%
Round 4
Global Model Accuracy: 49.00%
Round 5
Global Model Accuracy: 57.00%
Round 6
Global Model Accuracy: 44.00%
Round 7
Global Model Accuracy: 55.00%
Round 8
Global Model Accuracy: 59.00%
Round 9
Global Model Accuracy: 52.00%
Round 10
Global Model Accuracy: 50.00%


In [ ]:

Explanation
Federated Learning Process:
Each client trains a model locally on its own data.
Clients send model updates (parameters) to the central server.
The server aggregates these updates to improve the global model.
Differential Privacy in Aggregation:
Gaussian noise is added during the aggregation step (aggregate_models) to protect individual client contributions.
This ensures that individual updates cannot be reconstructed.
Synthetic Data:
Clients are simulated using randomly generated data and simple binary labels.
Global Model Evaluation:
After each federated learning round, the global model is evaluated on a synthetic test set.
Expected Output
The code will print the accuracy of the global model after each round. For example:
Round 1
Global Model Accuracy: 75.00%
Round 2
Global Model Accuracy: 80.00%
...

Key Concepts Demonstrated
Privacy-Preserving Federated Learning:
Local data stays on the client; only aggregated updates are shared.
Differential privacy ensures individual contributions are anonymized.
Decentralized Training:
Each client trains independently on its data, enabling learning across distributed datasets.
Scalable Aggregation:
The server efficiently combines updates, maintaining model performance while preserving privacy.




## RLHF with DPO


In [ ]:

import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np

# Define GPT-like model (based on Andrej Karpathy's GPT)
class GPT(nn.Module):
    def __init__(self, vocab_size, embed_size, num_heads, num_layers, hidden_dim, max_seq_len):
        super(GPT, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embed_size)
        self.positional_encoding = nn.Parameter(torch.randn(1, max_seq_len, embed_size))
        self.transformer = nn.TransformerEncoder(
            nn.TransformerEncoderLayer(embed_size, num_heads, hidden_dim),
            num_layers
        )
        self.fc = nn.Linear(embed_size, vocab_size)

    def forward(self, x):
        x = self.embedding(x) + self.positional_encoding[:, :x.size(1), :]
        x = self.transformer(x)
        return self.fc(x)

# Generate synthetic preference data
def generate_synthetic_data(batch_size, seq_len, vocab_size):
    # Generate random token sequences
    seq_a = torch.randint(0, vocab_size, (batch_size, seq_len))
    seq_b = torch.randint(0, vocab_size, (batch_size, seq_len))
    # Randomly assign preferences (1 means seq_a preferred over seq_b, 0 otherwise)
    preferences = torch.randint(0, 2, (batch_size,))
    return seq_a, seq_b, preferences

# Define reward model
class RewardModel(nn.Module):
    def __init__(self, embed_size, hidden_dim):
        super(RewardModel, self).__init__()
        self.fc = nn.Sequential(
            nn.Linear(embed_size, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, 1)
        )

    def forward(self, embeddings):
        return self.fc(embeddings).squeeze(-1)

# Calculate preference loss for DPO
def dpo_loss(reward_a, reward_b, preferences, beta=0.1):
    logits = (reward_a - reward_b) / beta
    loss = -torch.mean(preferences * torch.log_softmax(logits, dim=0))
    return loss

# Main training loop
def train_dpo(gpt_model, reward_model, optimizer_gpt, optimizer_reward, vocab_size, seq_len, epochs, batch_size):
    for epoch in range(epochs):
        # Generate synthetic data
        seq_a, seq_b, preferences = generate_synthetic_data(batch_size, seq_len, vocab_size)

        # Forward pass for both sequences
        logits_a = gpt_model(seq_a)
        logits_b = gpt_model(seq_b)

        # Reward computation
        reward_a = reward_model(logits_a.mean(dim=1))  # Mean embeddings
        reward_b = reward_model(logits_b.mean(dim=1))  # Mean embeddings

        # Compute DPO loss
        loss_dpo = dpo_loss(reward_a, reward_b, preferences)

        # Backpropagation for GPT model
        optimizer_gpt.zero_grad()
        loss_dpo.backward(retain_graph=True)
        optimizer_gpt.step()

        # Train reward model (optional)
        optimizer_reward.zero_grad()
        loss_dpo.backward()
        optimizer_reward.step()

        print(f"Epoch {epoch + 1}, DPO Loss: {loss_dpo.item():.4f}")

# Hyperparameters
vocab_size = 100  # Small vocab for synthetic data
embed_size = 128
num_heads = 4
num_layers = 2
hidden_dim = 256
max_seq_len = 32
seq_len = 16
batch_size = 32
epochs = 10
lr = 1e-3

# Initialize models and optimizers
gpt_model = GPT(vocab_size, embed_size, num_heads, num_layers, hidden_dim, max_seq_len)
reward_model = RewardModel(embed_size, hidden_dim)
optimizer_gpt = optim.Adam(gpt_model.parameters(), lr=lr)
optimizer_reward = optim.Adam(reward_model.parameters(), lr=lr)

# Train with DPO
train_dpo(gpt_model, reward_model, optimizer_gpt, optimizer_reward, vocab_size, seq_len, epochs, batch_size)

